In [1]:
!pip install catboost
!pip install tsfresh
!pip install xgboost
!pip install shap

In [2]:
# The essentials
import pandas as pd
import numpy as np

from collections import defaultdict

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt

# Progress bars
from tqdm import tqdm

# Access our Google Drive
from google.colab import drive

# Gradient Boosting
from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor

# TSFRESH Feature Extraction
from tsfresh import extract_features
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table

from sklearn.model_selection import KFold

from collections import defaultdict, Counter
from scipy.stats import norm

from scipy.stats import boxcox, boxcox_normmax
from scipy.special import inv_boxcox

from sklearn.preprocessing import PowerTransformer

import shap

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
drive.mount('/content/drive', force_remount=True)
!ls "/content/drive/My Drive/Rinse Over Run"

Mounted at /content/drive
20178.png
20451.png
20899.png
22112.png
22369.png
22414.png
22487.png
23011.png
23142.png
23599.png
23872.png
24804.png
24845.png
24872.png
25129.png
25908.png
25983.png
26270.png
27115.png
27243.png
27346.png
27366.png
27418.png
27508.png
all_train_preds_per_phase.p
baseline_features_with_preds_per_phase.csv
baseline_model_per_nunique_phases.csv
better_prev_object_id_per_10.csv
dtw_distances_3.p
extended_phase_predictors.csv
hcsta_features_3_3.csv
last_cleaned_test.csv
last_cleaned_train.csv
mds_embeddings_2d_3.csv
mds_embeddings_2d_3.p
model_per_recipe_simple.csv
more_features_with_preds_per_phase.csv
pca_features_with_preds_per_phase.csv
predictions_machine_405.csv
preds_feature_selection.csv
processes_all_phases.p
process_machine_outlier_predicted.csv
recipe_8_augmented.csv
recipe_metadata.csv
recipe_model_all_tsfresh_bitmaps.csv
recipe_model_prev_process_tsfresh.csv
stacking_v1.csv
supertest.csv
test_features_14.csv
test_features_15.csv
test_features_1.cs

In [4]:
train_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_values.csv', index_col=0, parse_dates=['timestamp'])
test_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/test_values.csv', index_col=0, parse_dates=['timestamp'])
label_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_labels.csv', index_col='process_id')
all_data = pd.concat([train_df, test_df], axis=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
train_df = train_df[train_df['phase'] != 'final_rinse']

train_df['phase_int'] = train_df['phase'].map({'pre_rinse': 1, 
                                               'caustic': 2, 
                                               'intermediate_rinse': 4, 
                                               'acid': 8})
test_df['phase_int'] = test_df['phase'].map({'pre_rinse': 1, 
                                             'caustic': 2, 
                                             'intermediate_rinse': 4, 
                                             'acid': 8})
train_process_combinations = pd.DataFrame(train_df.groupby('process_id')['phase_int'].unique().apply(lambda x: sum(x)))
test_process_combinations = pd.DataFrame(test_df.groupby('process_id')['phase_int'].unique().apply(lambda x: sum(x)))
process_combinations = pd.concat([train_process_combinations, test_process_combinations], axis=0)

In [0]:
recipe_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/recipe_metadata.csv', index_col='process_id')
recipe_df = recipe_df.drop('final_rinse', axis=1)
recipe_df['pre_rinse_num'] = recipe_df['pre_rinse'] * 1
recipe_df['caustic_num'] = recipe_df['caustic'] * 2
recipe_df['intermediate_rinse_num'] = recipe_df['intermediate_rinse'] * 4
recipe_df['acid_num'] = recipe_df['acid'] * 8
recipe_df['recipe'] = recipe_df['pre_rinse_num'] + recipe_df['caustic_num'] + recipe_df['intermediate_rinse_num'] + recipe_df['acid_num']

In [0]:
ts_real = [
    'supply_flow',
    'supply_pressure',
    'return_temperature',
    'return_conductivity',
    'return_turbidity',
    'return_flow',
    'tank_level_pre_rinse',
    'tank_level_caustic',
    'tank_level_acid',
    'tank_level_clean_water',
    'tank_temperature_pre_rinse',
    'tank_temperature_caustic',
    'tank_temperature_acid',
    'tank_concentration_caustic',
    'tank_concentration_acid',
    'target_value',
    'flow_diff',
    'supply_flow_log',
    'return_flow_log'
]

# variables we'll use to create our time series features
ts_cols = [
    'supply_flow',
    'supply_pressure',
    'return_temperature',
    'return_conductivity',
    'return_turbidity',
    'return_flow',
    'tank_level_pre_rinse',
    'tank_level_caustic',
    'tank_level_acid',
    'tank_level_clean_water',
    'tank_temperature_pre_rinse',
    'tank_temperature_caustic',
    'tank_temperature_acid',
    'tank_concentration_caustic',
    'tank_concentration_acid',
    'target_value',
    'flow_diff',
    #'supply_flow_log',
    #'return_flow_log'
]

# variables for binary time series features
bin_cols = [
    'supply_pump',
    'supply_pre_rinse',
    'supply_caustic',
    'return_caustic',
    'supply_acid',
    'return_acid',
    'supply_clean_water',
    'return_recovery_water',
    'return_drain',
    'object_low_level',
    'tank_lsh_caustic',
    'tank_lsh_acid',
    'tank_lsh_clean_water',
    'tank_lsh_pre_rinse'
]

process_comb_to_phases = {
    15: ['pre_rinse', 'caustic', 'intermediate_rinse', 'acid'],
    3:  ['pre_rinse', 'caustic'],
    7:  ['pre_rinse', 'caustic', 'intermediate_rinse'],
    1:  ['pre_rinse'],
    8:  ['acid'],
    2:  ['caustic'],
    6:  ['caustic', 'intermediate_rinse'],
    14: ['caustic', 'intermediate_rinse', 'acid'],
}

# phases, ordered from earliest to latest
phases = ['pre_rinse', 'caustic', 'intermediate_rinse', 'acid']

def encode_categorical(df):
    # Currently just copy-pasted from http://drivendata.co/blog/rinse-over-run-benchmark/
    
    # select process_id and pipeline
    meta = df[['process_id', 'pipeline', 'object_id']].drop_duplicates().set_index('process_id') 
    meta['object_id'] = meta['object_id'] // 10
    #meta = meta.merge(recipe_df[['recipe', 'pre_rinse', 'caustic', 'intermediate_rinse', 'acid']], left_index=True, right_index=True)
    #meta = meta.merge(process_combinations[['phase_int']], left_index=True, right_index=True)
    
    #meta = meta.merge(df.groupby('process_id')[['timestamp']].min(), left_index=True, right_index=True)
    
    # convert categorical pipeline data to dummy variables
    meta = pd.get_dummies(meta, columns=['pipeline', 'object_id'])
    
    # pipeline L12 not in test data (so useless feature)
    if 'pipeline_L12' in meta:
        meta = meta.drop('pipeline_L12', axis=1)
    
    #meta['dow'] = meta['timestamp'].apply(lambda x: x.weekday())
    #meta['hour'] = meta['timestamp'].apply(lambda x: x.hour)
    
    # calculate number of phases for each process_object
    #meta['num_phases'] = df.groupby('process_id')['phase'].apply(lambda x: x.nunique())
    #meta['object_id'] = df.groupby('process_id')['object_id'].median()
    
    #meta = meta.drop('timestamp', axis=1)
    
    return meta
  
def count_zeros(x):
  return np.sum(x == 0)
  
def encode_real_timeseries(df):   
    ts_df = df[['process_id'] + ts_cols].set_index('process_id')
    
    # create features: count, min, max, mean, standard deviation
    ts_features = ts_df.groupby('process_id').agg(['min', 'max', 'mean', 'std', 
                                                   'count', 'median', 'sum', 
                                                   lambda x: x.tail(5).mean(),
                                                   count_zeros])
    
    cols = []
    for col in ts_features.columns:
        cols.append('real_{}'.format(col))
    ts_features.columns = cols
    
    return ts_features

def encode_binary_timeseries(df):
    ts_df = df[['process_id'] + bin_cols].set_index('process_id')
            
    # create features: count, min, max, mean, standard deviation
    ts_features = ts_df.groupby('process_id').agg(['mean', 'std', 
                                                   lambda x: x.tail(5).mean(),
                                                   count_zeros])
    
    cols = []
    for col in ts_features.columns:
        cols.append('bin_{}'.format(col))
    ts_features.columns = cols
    
    return ts_features

def get_descript(data, functions, cols):
    ts_df = data.set_index('process_id').sort_values(by='timestamp')
    return ts_df.groupby('process_id')[cols].agg(functions)  

  
def get_descript_prev_process(data, data_procs):
    machines = set(data['object_id'])
    all_features = []
    for machine in tqdm(machines):
        machine_data = data[data['object_id'] == machine]
        machine_data = machine_data.sort_values(by='timestamp')
        machine_processes = machine_data['process_id'].unique()
        for process_ix, process in enumerate(machine_processes):
          if process in data_procs:
            if process_ix > 0:
                prev_process = machine_data[machine_data['process_id'] == machine_processes[process_ix - 1]]
                # TODO: Makes more sense to take first phase (since masking is done from the end)
                # this could still be another phase than pre_rinse though...
#                 first_phase = None
#                 for phase in phases:
#                   if phase in set(prev_process['phase']):
#                     first_phase = phase
#                     break
#                 prev_process = prev_process[prev_process['phase'] == first_phase]
                
#                 features = get_descript(prev_process, ['mean', 'std', 'min', 'max', 'sum'], ts_cols)
#                 _columns = list(features.columns)
#                 assert len(features) == 1
#                 features = features.iloc[0, :].values
                
                this_process = machine_data[machine_data['process_id'] == machine_processes[process_ix]]
                prev_recipe = recipe_df.loc[machine_processes[process_ix - 1]]['recipe']
#                 prev_phase_int = process_combinations.loc[machine_processes[process_ix - 1]]['phase_int']
                # From start to start, since these values are always provided (real end could be masked in test set)
                time_delta = (this_process['timestamp'].values[0] - prev_process['timestamp'].values[0]) / np.timedelta64(1, 'h')
                assert time_delta > 0
                all_features.append([machine, process, time_delta, prev_recipe])
            else:
                all_features.append([machine, process, np.NaN, np.NaN])
   
    all_features = pd.DataFrame(all_features, columns=['object_id', 'process_id', 'time_delta', 'recipe'])
    all_features = all_features.set_index('process_id', drop=True)
    col_map = {}
    for col in all_features.columns:
        col_map[col] = 'prev_{}'.format(col)
    all_features = all_features.rename(columns=col_map)
    all_features = all_features.drop('prev_object_id', axis=1)
    return all_features
  
def get_tsfresh_features(df):
    extraction_settings = EfficientFCParameters()
#     filtered_funcs = ['last_location_of_maximum', 'first_location_of_maximum', 'longest_strike_below_mean', 'last_location_of_minimum',
#                       'absolute_sum_of_changes', 'skewness', 'kurtosis', 'longest_strike_above_mean']
    filtered_funcs = ['abs_energy', 'mean_abs_change', 'mean_change', 
                      'skewness', 'kurtosis', 'absolute_sum_of_changes', 
                      'longest_strike_below_mean', 'longest_strike_above_mean', 
                      'count_above_mean', 'count_below_mean', 'last_location_of_maximum', 
                      'first_location_of_maximum', 'last_location_of_minimum', 
                      'first_location_of_minimum', 
                      'percentage_of_reoccurring_datapoints_to_all_datapoints', 
                      'percentage_of_reoccurring_values_to_all_values', 
                      'sum_of_reoccurring_values', 'sum_of_reoccurring_data_points', 
                      'ratio_value_number_to_time_series_length', 'maximum', 'minimum', 
                      'cid_ce', 'symmetry_looking', 'large_standard_deviation', 'quantile', 
                      'autocorrelation', 'number_peaks', 'binned_entropy', 'index_mass_quantile', 
                      'linear_trend',  'number_crossing_m']
    new_funcs = ['augmented_dickey_fuller', 'number_cwt_peaks', 'agg_autocorrelation',
               'spkt_welch_density', 'friedrich_coefficients', 'max_langevin_fixed_point',
               'c3', 'ar_coefficient', 'mean_second_derivative_central', 'ratio_beyond_r_sigma',
               'energy_ratio_by_chunks', 'partial_autocorrelation',
               'fft_aggregated', 'time_reversal_asymmetry_statistic', 'range_count']
    filtered_funcs += new_funcs
    filtered_settings = {}
    for func in filtered_funcs:
      filtered_settings[func] = extraction_settings[func]

    ts_features = extract_features(df[['process_id', 'timestamp', 'return_turbidity', 'return_flow', 'supply_flow', 'target_value', 'flow_diff']], 
                                   column_id='process_id', column_sort="timestamp", 
                                   column_kind=None, column_value=None,
                                   impute_function=impute, 
                                   default_fc_parameters=filtered_settings,
                                   show_warnings=False)
  
    return ts_features
                                       

def create_feature_matrix(df, processes, phases):
    df['return_flow'] = df['return_flow'].apply(lambda x: max(x, 0))
    df['supply_flow'] = df['supply_flow'].apply(lambda x: max(x, 0))
    df['target_value'] = df['return_flow'] * df['return_turbidity']
    df['flow_diff'] = df['supply_flow'] - df['return_flow']
    
    #df['return_flow_log'] = np.log1p(df['return_flow'])
    #df['supply_flow_log'] = np.log1p(df['supply_flow'])
    
    phase_data = df[(df['process_id'].isin(processes)) &
                    ((df['phase'].isin(phases)))]
    
    #prev_features = get_descript_prev_process(df, processes)
    #prev_features = pd.get_dummies(prev_features, columns=['prev_recipe'])
    
    metadata = encode_categorical(phase_data)
    time_series = encode_real_timeseries(phase_data)
    binary_features = encode_binary_timeseries(phase_data)
    
#     if len(phases) > 1:
#       last_phase_data = phase_data[phase_data['phase'] == phases[-1]]
#       time_series_last_phase = encode_real_timeseries(last_phase_data)
#       new_cols = []
#       for col in time_series_last_phase.columns:
#         new_cols.append('last_{}'.format(col))
#       time_series_last_phase.columns = new_cols
#       binary_features_last_phase = encode_binary_timeseries(last_phase_data)
#       new_cols = []
#       for col in binary_features_last_phase.columns:
#         new_cols.append('last_{}'.format(col))
#       binary_features_last_phase.columns = new_cols
    
#     tsfresh_features = get_tsfresh_features(phase_data)
    
    # join metadata and time series features into a single dataframe
    feature_matrix = metadata
    feature_matrix = feature_matrix.merge(time_series, left_index=True, right_index=True)
    feature_matrix = feature_matrix.merge(binary_features, left_index=True, right_index=True)
    #feature_matrix = feature_matrix.merge(prev_features, left_index=True, right_index=True)
#     feature_matrix = feature_matrix.merge(tsfresh_features, left_index=True, right_index=True)
    
#     if len(phases) > 1:
#       feature_matrix = feature_matrix.merge(time_series_last_phase, left_index=True, right_index=True)
#       feature_matrix = feature_matrix.merge(binary_features_last_phase, left_index=True, right_index=True)
      
#     transformer = PowerTransformer()
#     cols = feature_matrix.columns
#     indices = feature_matrix.index
#     feature_matrix = transformer.fit_transform(feature_matrix)
#     feature_matrix = pd.DataFrame(feature_matrix, index=indices, columns=cols)
    
    return feature_matrix
    
  
def get_processes(data, phases, train=True):
    filtered_processes = []
    phases = set(phases)
    processes = set(data['process_id'])
    for process in processes:
        process_phases = set(data[data['process_id'] == process]['phase'])
        if train:
            if phases.issubset(process_phases):
                filtered_processes.append(process)
        else:
            if len(phases) == len(process_phases) == len(phases.intersection(process_phases)):
                filtered_processes.append(process)
    return filtered_processes

In [0]:
#from tsfresh.feature_selection.relevance import calculate_relevance_table

def get_corr_features(X):
  row_idx, col_idx = np.where(X.corr() == 1)
  self_corr = set([(i, i) for i in range(X.shape[1])])
  return set(list(zip(row_idx, col_idx))) - self_corr 

def get_uncorr_features(data):
  X_train_corr = data.copy()
  correlated_features = get_corr_features(X_train_corr)
  
  corr_cols = set()
  for row_idx, col_idx in correlated_features:
    corr_cols.add(row_idx)
    corr_cols.add(col_idx)
  
  uncorr_cols = list(set(X_train_corr.columns) - set(X_train_corr.columns[list(corr_cols)]))
   
  col_mask = [False]*X_train_corr.shape[1]
  for col in corr_cols:
    col_mask[col] = True
  X_train_corr = X_train_corr.loc[:, col_mask]
  
  correlated_features = get_corr_features(X_train_corr)
  
  while correlated_features:
    print('{} correlated feature pairs left...'.format(len(correlated_features)))
    corr_row, corr_col = correlated_features.pop()
    col_mask = [True]*X_train_corr.shape[1]
    col_mask[corr_row] = False
    X_train_corr = X_train_corr.loc[:, col_mask]
    correlated_features = get_corr_features(X_train_corr)
  return list(set(list(X_train_corr.columns) + uncorr_cols))

def remove_features(data, target, p_val=0.25):
  single_cols = list(data.columns[data.nunique() == 1])
  
  uncorr_cols = get_uncorr_features(data)
  corr_cols = list(set(data.columns) - set(uncorr_cols))
  
  return list(set(single_cols + corr_cols))

In [0]:
def mean_absolute_percentage_error(y_true, y_pred, sample_weights=None):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    assert len(y_true) == len(y_pred)
    
    if np.any(y_true==0):
        print("Found zeroes in y_true. MAPE undefined. Removing from set...")
        idx = np.where(y_true==0)
        y_true = np.delete(y_true, idx)
        y_pred = np.delete(y_pred, idx)
        if type(sample_weights) != type(None):
            sample_weights = np.array(sample_weights)
            sample_weights = np.delete(sample_weights, idx)
    y_true = np.exp(y_true)
    y_pred = np.exp(y_pred)
    if type(sample_weights) == type(None):
        return(np.mean(np.abs((y_true - y_pred) / np.maximum(290000, y_true))))
    else:
        sample_weights = np.array(sample_weights)
        assert len(sample_weights) == len(y_true)
        return(sum(sample_weights)*np.dot(
                sample_weights, (np.abs((y_true - y_pred) / np.maximum(290000, y_true)))
        ))

In [0]:
from scipy.optimize import minimize

class MAPELinearModel:
    """
    Linear model: Y = XB, fit by minimizing the provided loss_function
    with L2 regularization
    """
    def __init__(self, loss_function=mean_absolute_percentage_error, 
                 X=None, Y=None, sample_weights=None, beta_init=None, 
                 regularization=0.001):
        self.regularization = regularization
        self.beta = None
        self.loss_function = loss_function
        self.sample_weights = sample_weights
        self.beta_init = beta_init
        
        self.X = X
        self.Y = Y
            
    
    def predict(self, X):
        prediction = np.matmul(X, self.beta)
        return(prediction)

    def model_error(self):
        error = self.loss_function(
            self.predict(self.X), self.Y, sample_weights=self.sample_weights
        )
        return(error)
    
    def l2_regularized_loss(self, beta):
        self.beta = beta
        return(self.model_error() + \
               sum(self.regularization*np.array(self.beta)**2))
    
    def fit(self, maxiter=2500):        
        # Initialize beta estimates (you may need to normalize
        # your data and choose smarter initialization values
        # depending on the shape of your loss function)
        if type(self.beta_init)==type(None):
            # set beta_init = 1 for every feature
            self.beta_init = np.random.random(self.X.shape[1])
        else: 
            # Use provided initial values
            pass
            
        if self.beta!=None and all(self.beta_init == self.beta):
            print("Model already fit once; continuing fit with more itrations.")
            
        res = minimize(self.l2_regularized_loss, self.beta_init,
                       method='BFGS', options={'maxiter': maxiter, 'disp': True})
        self.beta = res.x
        self.beta_init = self.beta
        
def custom_mape(approxes, targets):
    return np.mean(np.abs(np.subtract(approxes, targets)) / np.maximum(np.abs(targets), 290000))

In [0]:
from sklearn.base import clone

def fit_stack(clf, name, X_train, y_train, X_test, n_splits=5):
  train_predictions = np.zeros((len(X_train), 1))
  test_predictions = np.zeros((len(X_test), n_splits))
  kf = KFold(n_splits=n_splits)
  for fold_ix, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    X_cv_train = X.iloc[train_idx, :]
    X_cv_test = X.iloc[test_idx, :]
    y_cv_train = y.iloc[train_idx]
    y_cv_test = y.iloc[test_idx]
    
    clf_clone = clone(clf)
    clf_clone.fit(X_cv_train, y_cv_train)
    
    print('Fold #{} MAPE={}'.format(fold_ix + 1, custom_mape(np.exp(y_test), np.exp(clf_clone.predict(X_cv_test)))))
    
    train_predictions[train_idx] = clf_clone.predict(X_cv_test)
    test_predictions[:, fold_ix] = clf_clone.predict(X_test)
    
  train_predictions_df = pd.DataFrame(train_predictions, index=X_train.index, columns=['{}_pred'.format(name)])
  test_predictions_df = pd.DataFrame(np.mean(test_predictions, axis=1), index=X_test.index, columns=['{}_pred'.format(name)])
    
  return train_predictions_df, test_predictions_df

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor

combinations_per_recipe = {
    3: [3],  # 2 not included
    9: [8],
    15: [3, 7, 15]
}

prediction_df = None
all_mapes = defaultdict(list)
for recipe in [3, 9, 15]:
    recipe_train_data = train_df[train_df['process_id'].isin(recipe_df[recipe_df['recipe'] == recipe].index)]
    recipe_test_data = test_df[test_df['process_id'].isin(recipe_df[recipe_df['recipe'] == recipe].index)]
    for process_combination in combinations_per_recipe[recipe]:
      print('Recipe = {} || Combination = {}'.format(recipe, process_combination))
      train_processes = get_processes(recipe_train_data, process_comb_to_phases[process_combination])
      phase_features = create_feature_matrix(train_df, train_processes, process_comb_to_phases[process_combination])
      
      X = phase_features.loc[train_processes]
      y = np.log(label_df.loc[X.index]['final_rinse_total_turbidity_liter'])
    
      to_drop = remove_features(X, y)
      X = X.drop(to_drop, axis=1)

      kf = KFold(n_splits=5, random_state=2019)
      mapes = []
      for train_idx, test_idx in kf.split(X, y):
        X_train = X.iloc[train_idx, :]
        X_test = X.iloc[test_idx, :]

        print('Transforming data...')
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        y_train = y.iloc[train_idx]
        y_test = y.iloc[test_idx]

        print('Fitting model...')
        lr = MAPELinearModel(X=X_train, Y=y_train)#GridSearchCV(MLPRegressor(), {'hidden_layer_sizes': [(100,), (250,), (100, 100)]})
        lr.fit()
        #print(lr.best_params_)

        print(list(lr.predict(X_test)))

        preds = np.exp(np.minimum(np.max(y_train), np.maximum(0, lr.predict(X_test))))
        mape = custom_mape(preds, np.exp(y_test))
        print('TEST MAPE = {}'.format(mape))
        mapes.append(mape)
      
      print('Combination = {}, MAPE = {}+-{}'.format(process_combination, np.mean(mapes), np.std(mapes)))
   

Recipe = 3 || Combination = 3
480 correlated feature pairs left...
478 correlated feature pairs left...
436 correlated feature pairs left...
396 correlated feature pairs left...
358 correlated feature pairs left...
322 correlated feature pairs left...
288 correlated feature pairs left...
256 correlated feature pairs left...
226 correlated feature pairs left...
198 correlated feature pairs left...
172 correlated feature pairs left...
148 correlated feature pairs left...
126 correlated feature pairs left...
106 correlated feature pairs left...
88 correlated feature pairs left...
72 correlated feature pairs left...
58 correlated feature pairs left...
56 correlated feature pairs left...
44 correlated feature pairs left...
34 correlated feature pairs left...
26 correlated feature pairs left...
20 correlated feature pairs left...
18 correlated feature pairs left...
14 correlated feature pairs left...
10 correlated feature pairs left...
8 correlated feature pairs left...
6 correlated feature 

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.


[55.54922797882284, -24.406113839503234, -37.85298206662779, 47.83506353141219, -117.73741623082164, -8.688571994009214, -126.25936251194929, 56.89830230713835, 8.937662392761915, -103.61780932225227, 17.25688842103173, 10.607638730388077, 7.330222888845003, -32.37611548535389, 109.68514010763224, -13.875076949141544, 49.32198457195595, -95.87648500528789, 10.849847785983181, 125.9283766310557, -7.809174570669617, 70.55275670930428, 94.41692289606674, -25.40258725395001, 81.32299156489871, 23.739591966617166, 6.634557943732778, 29.587092528395956, 222.9452239681025, -77.13876948141456, 21.727508278579478, 5.811169888843319, -101.1658705906164, -106.61516420995963, -68.18567249557738, -71.47843779334926, 34.322343457361605, 44.41258739836249, 31.578487578362317, 122.82186261914944, -125.11454685432035, 20.85355363578466, 35.54326948214738, 33.415351544275936, -34.39605333901921, -132.7812801640823, 65.41784363772288, 57.65705395385807, -68.31006312111163, -124.77275317342239, 86.1195717

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.


KeyboardInterrupt: ignored

In [47]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

combinations_per_recipe = {
    3: [3],  # 2 not included
    9: [8],
    15: [3, 7, 15]
}

prediction_df = None
all_mapes = defaultdict(list)
for recipe in [3, 9, 15]:
    recipe_train_data = train_df[train_df['process_id'].isin(recipe_df[recipe_df['recipe'] == recipe].index)]
    recipe_test_data = test_df[test_df['process_id'].isin(recipe_df[recipe_df['recipe'] == recipe].index)]
    for process_combination in combinations_per_recipe[recipe]:
      print('Recipe = {} || Combination = {}'.format(recipe, process_combination))
      train_processes = get_processes(recipe_train_data, process_comb_to_phases[process_combination])
      phase_features = create_feature_matrix(train_df, train_processes, process_comb_to_phases[process_combination])
      
      X = phase_features.loc[train_processes]
      y = np.log(label_df.loc[X.index]['final_rinse_total_turbidity_liter'])
    
      to_drop = remove_features(X, y)
      X = X.drop(to_drop, axis=1)
      
      kf = KFold(n_splits=5, random_state=2019)
      mapes = []
      for train_idx, test_idx in kf.split(X, y):
        X_train = X.iloc[train_idx, :]
        X_test = X.iloc[test_idx, :]
  
        print('Transforming data...')
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        y_train = y.iloc[train_idx]
        y_test = y.iloc[test_idx]

        print('Fitting model...')
        lr = GridSearchCV(Ridge(), {'alpha': [10.0, 100.0, 1000.0]})
        lr.fit(X_train, y_train)
        print(lr.best_params_)

        print(list(lr.predict(X_test)))

        preds = np.exp(np.minimum(np.max(y_train), np.maximum(0, lr.predict(X_test))))
        mape = custom_mape(preds, np.exp(y_test))
        print('TEST MAPE = {}'.format(mape))
        mapes.append(mape)
      
      print('Combination = {}, MAPE = {}+-{}'.format(process_combination, np.mean(mapes), np.std(mapes)))
   

Recipe = 3 || Combination = 3
480 correlated feature pairs left...
478 correlated feature pairs left...
436 correlated feature pairs left...
396 correlated feature pairs left...
358 correlated feature pairs left...
322 correlated feature pairs left...
288 correlated feature pairs left...
256 correlated feature pairs left...
226 correlated feature pairs left...
198 correlated feature pairs left...
172 correlated feature pairs left...
148 correlated feature pairs left...
126 correlated feature pairs left...
106 correlated feature pairs left...
88 correlated feature pairs left...
72 correlated feature pairs left...
58 correlated feature pairs left...
56 correlated feature pairs left...
44 correlated feature pairs left...
34 correlated feature pairs left...
26 correlated feature pairs left...
20 correlated feature pairs left...
18 correlated feature pairs left...
14 correlated feature pairs left...
10 correlated feature pairs left...
8 correlated feature pairs left...
6 correlated feature 

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Dat

{'alpha': 100.0}
[13.915234114777741, 15.270798057545727, 14.934372025937392, 14.543300446180305, 14.111472703559217, 12.121425775956986, 14.653652181726784, 14.475646140023573, 15.446396448286297, 14.672916452812306, 12.507810440707518, 12.322349818488435, 13.555126423373242, 12.613888330805876, 14.521925975798176, 11.757440261506833, 15.133841824379868, 14.896507635042667, 14.045798328200773, 15.148559462628745, 14.434200341622509, 15.753309188782435, 15.499523621897284, 13.61374115921149, 15.768183714496459, 15.071365404755742, 14.627702789591888, 14.188104215233217, 15.864221960295097, 12.89511393994294, 14.646571209473809, 12.583921851182831, 13.866706313202537, 13.543850015697544, 14.71153332011206, 15.185085220169857, 13.58724492540151, 14.71674816904863, 14.60586024101562, 14.942053487243864, 14.146906336244502, 13.41235392798504, 13.309132046256298, 13.578470281144664, 15.329302583568948, 15.065631860710846, 14.917170208791026, 13.886393295673761, 13.800417044654658, 14.312305

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Dat

{'alpha': 100.0}
[15.55467533248836, 14.105300678288788, 14.391941100986537, 13.601840939773476, 13.984932952336411, 14.240700302406543, 15.282878347053815, 14.893878119130559, 13.673385715937018, 15.180941659351399, 15.343597325151112, 15.021546129025335, 15.546323543556866, 14.353926427149274, 13.515757680371847, 14.731501195785535, 14.402549848051756, 13.31192938136508, 14.787837367264999, 14.034534025220754, 14.62924654874485, 14.975738550204047, 14.772402603673964, 13.814403654550206, 15.187324465684181, 14.516035134006346, 15.252914493766692, 15.220903747512715, 14.263955446898951, 13.551190556244181, 13.90530864970073, 14.544699288763645, 14.54504814033863, 15.10029201273916, 14.899612231148353, 14.886599670667104, 15.286806475324004, 15.48628280634396, 13.808319480613845, 15.508591143481539, 15.451580322100344, 13.677402774057786, 13.775280354298248, 13.970485420655214, 14.693819975692719, 16.216482735412846, 14.655701389431826, 13.833318258600736, 12.320704840386949, 14.404658

...

{'alpha': 100.0}
[13.363289300402306, 13.611729922101658, 13.93976045316653, 13.89016392392907, 15.750710617508096, 13.480808557382286, 11.539114538172424, 12.724535353570808, 13.81225490237113, 12.835129441045117, 12.333870569230207, 13.991936834837562, 13.832363422830591, 13.147134549253312, 13.405898451170442, 15.035700335954967, 15.906156745635323, 13.811905903388663, 13.550202124480256, 11.234261916367018, 15.544270799907974, 15.16411075129464, 13.160432716355741, 15.56039963825498, 14.168609181488472, 12.500346888070853, 13.720871547843329, 12.33738594161043, 14.023015985682388, 13.630379030673955, 13.518750067662381, 13.97182236965083, 11.527632719685, 15.47760656827089, 10.490084155434223, 13.396235582957889, 15.271174895751276, 13.05461549486058, 13.576164041123159, 14.005496400591554, 15.040750231981521, 12.16707014257871, 13.847176802760572, 12.878693754155586, 13.919505777348318, 15.596319828166465, 15.495134053458099, 12.614926517706465, 13.50808709444954, 12.2077374540978

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'alpha': 100.0}
[11.821673482469118, 10.459630336495719, 13.819976944991826, 14.307752735233358, 15.476396255661422, 13.262531552750787, 13.673346491358725, 12.90500033555321, 14.072035017247675, 12.604286181872254, 13.715871971999489, 12.904981706172514, 13.615676100715808, 13.33065255340369, 14.212427574221396, 11.369171984482977, 12.755030504292954, 15.401491935938786, 13.42357109584611, 11.287504480243674, 12.918047875916843, 13.117208851746573, 14.837269183076458, 12.243949938061457, 11.93407498050803, 13.920699947998557, 13.145497229835536, 14.118300795544988, 13.774674368270569, 12.958367932954747, 10.347736064950054, 11.324127253477165, 12.568816438853675, 14.578609878848258, 15.481559869509619, 15.459442838296818, 14.590730295701153, 12.10793055134395, 10.343177388364925, 13.497449673290225, 13.367032664064205, 15.46365719547373, 13.948976823788863, 13.599658431042354, 14.405211658696434, 12.5142695135399, 15.428730858339158, 12.175949395668058, 12.939843663742842, 13.9260320

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'alpha': 100.0}
[11.715512190728356, 13.953228061991947, 13.268414524746543, 13.024418293721782, 13.2703267537612, 15.09168802200107, 13.657216574903115, 13.419598662497958, 12.451664502364642, 14.977632863269791, 10.57794708851669, 10.68122421892185, 13.198655227638517, 14.231571249168013, 11.448226849458804, 12.88913665212574, 13.947751915165457, 14.129037662604869, 13.455513214463167, 13.894075568138504, 11.685648525305075, 13.493882845070393, 15.63453603200507, 12.411038128791446, 11.751167443299881, 12.351069721981334, 13.918276542787012, 15.363118697094013, 13.655168145241769, 16.26890947231876, 13.156189280149706, 13.09376393636701, 13.355201383778896, 13.940763920211419, 13.426142332877163, 13.42273578569111, 13.537796570315251, 13.768531830745475, 10.98629799778114, 12.27627747017052, 13.350797820384658, 14.467889905804554, 13.541506610931451, 15.49606777769136, 14.154961788148578, 13.101842295422458, 13.69720296902586, 13.28249054280447, 13.263154100962419, 14.05255534495409

In [44]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

kf = KFold(n_splits=5, random_state=2019)
mapes = []
for train_idx, test_idx in kf.split(X, y):
  X_train = X.iloc[train_idx, :]
  X_test = X.iloc[test_idx, :]
  
  print('Transforming data...')
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  y_train = y.iloc[train_idx]
  y_test = y.iloc[test_idx]

  print('Fitting model...')
  lr = GridSearchCV(Ridge(), {'alpha': [10.0, 100.0, 1000.0]})
  lr.fit(X_train, y_train)
  print(lr.best_params_)
 
  print(list(lr.predict(X_test)))

  preds = np.exp(np.minimum(np.max(y_train), np.maximum(0, lr.predict(X_test))))
  mape = custom_mape(preds, np.exp(y_test))
  print('TEST MAPE = {}'.format(mape))
  mapes.append(mape)
  


Fitting model...
{'alpha': 1000.0}
[14.054294822941628, 14.909412224031911, 15.035696380765101, 14.43386487415646, 14.200264760285819, 12.182830132194479, 14.946244296700888, 14.400437358771493, 15.497777925783616, 14.852809886992322, 12.212259911501453, 12.222159742961555, 12.748257612114724, 12.290342197474175, 14.337404807482285, 11.731804817804475, 15.036952777508505, 14.790221339415965, 14.133988810128551, 15.162041409543056, 14.226282492384335, 16.036715812977633, 15.820999849867807, 13.74952629057167, 16.10392658325991, 15.138760310189227, 14.563268306493255, 14.175089854298761, 17.23062809193621, 13.570343597928385, 14.86576459671159, 12.284815953382305, 13.504919320150243, 13.471759118706938, 14.967964718910556, 15.002138408271112, 13.693573051258436, 15.055720627221207, 14.613129760589535, 14.767653787463058, 14.108638460778508, 13.092022894380719, 12.952129328960487, 13.613061674783346, 15.4057645692333, 14.898094674541568, 14.75236025525964, 13.917698381724014, 13.923909817

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.408441e-19
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.092117e-19
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.038529e-18
  overwrite_a=Tru

{'alpha': 100.0}
[14.5251595559225, 13.304223960332031, 14.07885997435262, 14.953088725116551, 14.645185270832389, 11.799007793489636, 11.91096923730186, 13.270861798990774, 14.270716230033054, 13.75052574019419, 20.918563724999345, 15.525456017538412, 15.647316545624154, 14.484040479315953, 13.3104397628387, 13.776670310205644, 13.504402680290747, 13.833746005390344, 14.468262855606236, 14.21352983682471, 14.795619972676475, 14.80560027019109, 14.229879159041676, 14.21217359517959, 13.947422898174196, 14.820252968007756, 13.664857894881338, 14.333959536067143, 12.883449797754686, 13.544719117454292, 15.610275778200993, 15.540161555096052, 14.167349774973776, 13.780890174962462, 14.268306996708048, 13.693510516430615, 12.868345645673894, 13.542579581446661, 13.942929069874019, 12.50912214661585, 14.079364631367365, 13.72287683139842, 14.032339623579118, 13.487840594319112, 13.510217254892769, 14.350202948620618, 16.102433466927856, 13.18157239355118, 14.731666581679237, 15.551957859502

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.590619e-19
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.021458e-17
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.074869e-18
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number8.667852e-18
  overwrite_a=True).T
/usr/local/lib/python3.6

{'alpha': 100.0}
[15.79077784942955, 14.021668166535001, 14.194968954078066, 13.378613800559018, 14.09028831610459, 14.110314309744636, 15.057034676600974, 15.03595562288282, 13.762895786863744, 15.068148690387599, 15.267717612713097, 14.844222452171067, 15.508301528388861, 13.899113064536223, 13.24577970538792, 14.486442265015544, 14.485463876507428, 13.185139042775397, 14.658844923386258, 14.066808991359341, 14.540388518469895, 15.505335802846096, 14.766703115313252, 13.80018584667214, 15.074939346283896, 14.578430723981247, 15.368747845015477, 15.43074118802281, 14.09249520235108, 13.65320733024072, 14.083592218432171, 14.651810673808466, 14.565068437348001, 15.013375288577786, 14.742028380050987, 14.990708606394131, 15.737898582527583, 15.6079310799174, 13.890722635558783, 14.99161980995489, 15.601905599661556, 13.688594930747346, 13.614977704102785, 13.850000616230336, 14.712657981770922, 21.75933258720573, 14.661694145524406, 13.730045791914819, 12.266562639566033, 13.26192215543

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number7.558810e-18
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number8.938453e-17
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number8.779893e-17
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number8.489270e-17
  overwrite_a=True).T
/usr/local/lib/python3.6

{'alpha': 1000.0}
[13.03361478865871, 13.854269739072143, 14.034080613356743, 13.96559537947065, 14.004229289600465, 14.455861802111045, 13.58905011213448, 13.584165958831202, 15.304690156818143, 13.64483095874144, 15.955886243800526, 14.933095154504121, 15.137478120197816, 14.111376178213968, 15.091385530155563, 14.474933680830112, 13.23495766698224, 14.877091197891989, 14.163595957366603, 14.185636059389168, 13.35299648554941, 15.939501767408847, 13.912937281511239, 12.2742712593855, 13.895480807268134, 15.902438461909636, 14.18797819910325, 15.301524088313169, 13.984118308205856, 14.343460488782224, 14.068261038511045, 13.683786615153576, 14.821190645668205, 14.738864681870696, 14.434590326053485, 14.553078444668303, 14.543735046727296, 14.226738080228552, 14.859683626595348, 14.76746658990535, 13.903399528123954, 15.082581943028128, 14.649469929929841, 15.449613649133255, 14.105706869528735, 14.922523328559794, 15.520817220655784, 13.606271209247964, 15.449360608025696, 13.50943931

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.039746e-17
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number8.489270e-17
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number6.441042e-17
  overwrite_a=True).T


In [83]:
from autograd import grad
import autograd.numpy as np

#y = y.values
#X = X.values.astype(float)

# scaler = StandardScaler()
# X = scaler.fit_transform(X)
#y = (y - np.mean(y)) / np.std(y)

def wTx(w, x):
   return np.dot(x, w)
  
def custom_loss(y, y_predicted):
    return np.mean(np.abs(np.subtract(y_predicted, y)) / np.abs(y))

def custom_loss_given_weights(w, batch_size=16):
    sample = np.random.choice(range(len(X)), replace=False, size=batch_size)
    y_predicted = wTx(w, X[sample, :])
    return custom_loss(y[sample], y_predicted) + np.sum(np.power(w, 2))
  
weights = np.zeros(X_train.shape[1])
eps = 1e-6
gradient = grad(custom_loss_given_weights)

for i in range(100000):
    if i % 1000 == 0:
        print('Iteration %-4d | Loss: %.4f' % (i, custom_loss_given_weights(weights)))
        print(weights[:5], gradient(weights)[:5])
    weights -= gradient(weights) * .01

Iteration 0    | Loss: 1.0000
[0. 0. 0. 0. 0.] [-1.404144   -1.261959   -1.28661859 -0.19756962 -2.01096139]
Iteration 1000 | Loss: 93.6338
[-0.26072006  0.31465432 -0.1420654   0.11980074  0.19058008] [-4.91480763 -2.55124623  1.61396391 -1.60306379 -2.92045681]
Iteration 2000 | Loss: 34.7640
[ 0.07663815 -0.0529307  -0.25919485  0.02761235 -0.07597725] [-1.29126202 -0.59647319 -0.61273806 -0.0386241  -2.3390103 ]
Iteration 3000 | Loss: 32.1331
[ 0.18136292  0.11585243  0.13284038  0.19137182 -0.49986158] [ 5.73731081 -0.30717505  0.43210905  0.54829077 -0.9401858 ]
Iteration 4000 | Loss: 34.7888
[ 0.0544033   0.1759061  -0.34349669 -0.00476784 -0.05561419] [ 2.44154207  1.4825385  -3.01294659  1.30630557 -7.50514131]
Iteration 5000 | Loss: 228.9142
[0.34444874 0.24988231 0.88065501 0.24565452 0.65357602] [ 1.74937352  0.75466596 -0.63214368  0.76621024  2.81614259]
Iteration 6000 | Loss: 96.1119
[ 0.45625136 -0.12795992 -0.24038085 -0.27093643  0.61793519] [ 1.55201332 -0.23756388 -0

KeyboardInterrupt: ignored

In [95]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X.shape[1]))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1))

model.compile(loss='mean_absolute_percentage_error',
              optimizer=Adam(), metrics=['mean_absolute_percentage_error'])

model.fit(X, y, batch_size=16, verbose=1, epochs=1000)

Epoch 1/1000
3726/3726 [==============================] - 1s 347us/step - loss: 437.4084 - mean_absolute_percentage_error: 437.4084
Epoch 2/1000
3726/3726 [==============================] - 1s 247us/step - loss: 397.8028 - mean_absolute_percentage_error: 397.8028
Epoch 3/1000
3726/3726 [==============================] - 1s 244us/step - loss: 276.6109 - mean_absolute_percentage_error: 276.6109
Epoch 4/1000
3726/3726 [==============================] - 1s 237us/step - loss: 305.6777 - mean_absolute_percentage_error: 305.6777
Epoch 5/1000
3726/3726 [==============================] - 1s 243us/step - loss: 214.9634 - mean_absolute_percentage_error: 214.9634
Epoch 6/1000
3726/3726 [==============================] - 1s 246us/step - loss: 241.0881 - mean_absolute_percentage_error: 241.0881
Epoch 7/1000
3726/3726 [==============================] - 1s 247us/step - loss: 209.8362 - mean_absolute_percentage_error: 209.8362
Epoch 8/1000
3726/3726 [==============================] - 1s 245us/step - lo

KeyboardInterrupt: ignored

In [0]:
from sklearn.neural_network import MLPRegressor

In [3]:
from sklearn.neural_network import MLPRegressor

kf = KFold(n_splits=5, random_state=2019)
mapes = []
for train_idx, test_idx in kf.split(X, y):
  X_train = X.iloc[train_idx, :]
  X_test = X.iloc[test_idx, :]
  
  print('Transforming data...')
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  y_train = y.iloc[train_idx]
  y_test = y.iloc[test_idx]

  print('Fitting model...')
  lr = GridSearchCV(MLPRegressor(), {'hidden_layer_sizes': [(100,), (250,), (100, 100)]})
  lr.fit(X_train, y_train)
  print(lr.best_params_)
 
  print(list(lr.predict(X_test)))

  preds = np.exp(np.minimum(np.max(y_train), np.maximum(0, lr.predict(X_test))))
  mape = custom_mape(preds, np.exp(y_test))
  print('TEST MAPE = {}'.format(mape))
  mapes.append(mape)
  


NameError: ignored

In [68]:
X = phase_features.loc[train_processes]
y = np.log(label_df.loc[X.index]['final_rinse_total_turbidity_liter'])

to_drop = remove_features(X, y)
X = X.drop(to_drop, axis=1)

kf = KFold(n_splits=5, random_state=2019)
mapes = []
for train_idx, test_idx in kf.split(X, y):
  X_train = X.iloc[train_idx, :]
  X_test = X.iloc[test_idx, :]

  print('Transforming data...')
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  y_train = y.iloc[train_idx]
  y_test = y.iloc[test_idx]

  print('Fitting model...')
  lr = MAPELinearModel(X=X_train, Y=y_train, regularization=10.)#GridSearchCV(MLPRegressor(), {'hidden_layer_sizes': [(100,), (250,), (100, 100)]})
  lr.fit(maxiter=2500)
  #print(lr.best_params_)

  print(list(lr.predict(X_test)))

  preds = np.minimum(np.max(np.exp(y_train)), np.maximum(0, np.exp(lr.predict(X_test))))
  mape = custom_mape(preds, np.exp(y_test))
  print('TEST MAPE = {}'.format(mape))
  mapes.append(mape)

print('Combination = {}, MAPE = {}+-{}'.format(process_combination, np.mean(mapes), np.std(mapes)))

480 correlated feature pairs left...
478 correlated feature pairs left...
436 correlated feature pairs left...
396 correlated feature pairs left...
358 correlated feature pairs left...
322 correlated feature pairs left...
288 correlated feature pairs left...
256 correlated feature pairs left...
226 correlated feature pairs left...
198 correlated feature pairs left...
172 correlated feature pairs left...
148 correlated feature pairs left...
126 correlated feature pairs left...
106 correlated feature pairs left...
88 correlated feature pairs left...
72 correlated feature pairs left...
58 correlated feature pairs left...
56 correlated feature pairs left...
44 correlated feature pairs left...
34 correlated feature pairs left...
26 correlated feature pairs left...
20 correlated feature pairs left...
18 correlated feature pairs left...
14 correlated feature pairs left...
10 correlated feature pairs left...
8 correlated feature pairs left...
6 correlated feature pairs left...
4 correlated fea

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()


         Current function value: 571.728178
         Iterations: 227
         Function evaluations: 58506
         Gradient evaluations: 325
[14.340352591839775, -22.70175682668618, 11.562139401405698, 14.172117562496709, -32.472607533737154, -6.845776208289674, -26.74804162235291, 15.691805538305239, 16.071299831951865, -27.19307739374601, -5.670502353006098, -5.601128290900246, -27.064263508225356, -7.542302493475818, 26.575847256901977, -9.406243708705352, 16.817979031341682, -26.810476066585988, 16.412688488964413, 26.937158970713423, 13.600778109861265, 21.06997486578682, 25.683438457898685, 11.35288772572743, 23.84424874051301, 13.872405979236579, 13.598896798339343, 15.373541582651182, 29.15156742043329, -12.66991822920385, 14.569633256642287, -5.387178472547147, -19.117682916388446, -37.110205710538175, -24.896254775989778, -25.72991330337528, 13.190517497585324, 14.473818953032863, 12.807203649885215, 26.531470857082198, -32.57348226879818, 13.688653638293793, 15.5341253369616

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()


         Current function value: 571.447214
         Iterations: 161
         Function evaluations: 49512
         Gradient evaluations: 275
[7.624029340078229, -1.0381759334129166, -20.30515834097727, 15.820044444764145, 15.4108256410079, -1.3504594925625348, -0.6593338481278929, -3.358304311220304, 9.76325288330183, 1.321633526035714, 46.18877988211725, 18.034263044765144, 23.410817702717114, 2.0052261816470183, -21.400765631327108, -20.262434280891934, 1.4424467923265112, -1.5354083093419153, 24.02718776226828, 13.920526475958885, 15.805874340894054, 15.849826368430387, 0.6865636170316312, -21.798713942860857, -3.386258129474361, 15.630550513501799, -3.4567411152014977, -19.393411233775154, -5.064490706473364, -1.026585087353577, 23.28140885025984, 17.495840477349947, -19.43391982103297, -0.2600505304279609, 15.13279745262302, -1.870609065689766, -3.7716158479816846, -23.32527293775933, -20.481553872107003, 78.27012532893546, 14.933782248251891, -2.465721091169955, -0.17758955148523

...

         Current function value: 556.731703
         Iterations: 87
         Function evaluations: 34932
         Gradient evaluations: 194
[19.67560427010086, 13.730661400600821, 14.458784819006164, 14.661550868175564, -5.362690571245942, 13.560756955388552, 21.22446303409124, 15.351177369621364, 13.988142499183697, 17.076904775866357, 20.670083711416275, 17.18251952586712, 15.893973010914781, -15.177729198805215, 14.908132670452682, 14.025884696274732, -29.555775936685297, 14.038188705222982, 14.44168752826343, 12.71032073109433, 15.552220886235231, 28.403459652086053, 16.133785717978608, 13.622016234266592, -21.04688201275185, -21.280653552453334, -24.78463451173162, 21.52653991902474, 13.92324194838317, 13.899310023227418, -28.888849260652506, -21.84464099332883, 17.227547460950404, 13.967138965608802, 17.371272394960744, -21.73988993288583, 18.853569902064507, -25.76286653948998, 13.228446859223892, -28.13036544445289, 18.41810781764603, 12.995342157082188, 11.867748105165848, 12.

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()


         Current function value: 576.118618
         Iterations: 182
         Function evaluations: 52209
         Gradient evaluations: 290
[14.489799449199152, -4.530462828696252, 22.333441776932872, -1.6516188801084373, -1.9084265776834974, -17.924968024698916, 22.38564748753855, -16.18931574937848, -8.861820878030318, -20.85553821152195, -2.974354393533828, -22.47016755097734, 14.286700021161366, -21.395072634486265, 16.13450233936183, -20.95478328882331, 14.174183311100979, 23.259409514292784, -2.30713728726829, -1.2963147748365795, -22.386397096235864, -9.239102228207859, -20.85402637529515, -18.41882324587546, -1.2869628881469597, -22.836631474651767, -21.430074140383795, -0.5619943537479278, -1.7901037284633217, 13.12815713193048, 14.94938936694312, -3.0693629261360265, -2.3565054464932302, 23.22209840572574, 23.048123427779522, -2.705079306081897, 12.655771778480549, -19.956734757027924, -3.113070876452875, -1.5297426368142193, -1.4053278175561856, -15.615044474123858, 26.0743

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()


         Current function value: 563.248063
         Iterations: 137
         Function evaluations: 42492
         Gradient evaluations: 236
[15.29212972218675, 13.058582389230258, -32.06125436154524, -33.02505681668481, 12.94203151247676, 24.173879751299506, 12.978541872777981, -38.87087597190073, -27.076446087173053, 12.171270314261712, 17.391917025712903, 15.578633906187699, 15.981494968423682, -29.229133708448373, 12.014187097412586, -30.861943377161488, 15.496172710846448, -23.50770744409913, -15.545614549422863, -31.654217573426955, -37.970355432937424, 26.524685766338674, 12.923071699945416, -9.381284394748704, 13.748351135127706, 25.79723454682843, -29.694323907404517, -28.801574676044012, -34.528080667992455, 15.239097228044292, -31.08887118139987, 12.167934662280162, -25.947409155575663, 14.31425646539205, 15.449807398947222, 16.33073950883355, -28.60955860745278, 12.60320035044917, 18.176631675531084, 16.77910135658809, 13.000786009997935, 21.585702665447737, 25.125525720115

In [65]:
mean_absolute_percentage_error(y_test, np.log(preds))

108.38830874730294